# Линейная регрессия: прогноз оклада по описанию вакансии
1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).
2. Проведите предобработку:
- Приведите тексты к нижнему регистру (text.lower()).
- Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты.

- Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
- Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.
- Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
- Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.
3.  Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

4.  Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [111]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge                 # гребневая регрессия
from sklearn.feature_extraction import DictVectorizer  # for nominal features (LocationNormalized, ContractTime)
from scipy.sparse import hstack

In [103]:
train = pd.read_csv('data/salary-train.csv')
X_train = train.iloc[:,:3]
y_train = train.iloc[:,3]

test = pd.read_csv('data/salary-test-mini.csv')
X_test = test.iloc[:,:3]
y_test = test.iloc[:,3]

X_train.iloc[:,0], X_test.iloc[:,0]

(0        International Sales Manager London ****k  ****...
 1        An ideal opportunity for an individual that ha...
 2        Online Content and Brand Manager// Luxury Reta...
 3        A great local marketleader is seeking a perman...
 4        Registered Nurse / RGN  Nursing Home for Young...
                                ...                        
 59995    As a result of continued growth, First Class S...
 59996    PHP / MVC Web Developer  MacclesfieldCirca ***...
 59997    Staff Nurse, Nursing Home, Baldock White Recru...
 59998    This is one of the best agency side opportunit...
 59999    Must have CSCS card must have asbestos awarene...
 Name: FullDescription, Length: 60000, dtype: object,
 0    We currently have a vacancy for an HR Project ...
 1    A Web developer opportunity has arisen with an...
 Name: FullDescription, dtype: object)

In [104]:
X_train['FullDescription'] = X_train['FullDescription'].apply(lambda x: x.lower())
X_train['FullDescription'] = X_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

X_test['FullDescription'] = X_test['FullDescription'].apply(lambda x: x.lower())
X_test['FullDescription'] = X_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

X_train['FullDescription'], X_test['FullDescription']

(0        international sales manager london     k      ...
 1        an ideal opportunity for an individual that ha...
 2        online content and brand manager   luxury reta...
 3        a great local marketleader is seeking a perman...
 4        registered nurse   rgn  nursing home for young...
                                ...                        
 59995    as a result of continued growth  first class s...
 59996    php   mvc web developer  macclesfieldcirca    ...
 59997    staff nurse  nursing home  baldock white recru...
 59998    this is one of the best agency side opportunit...
 59999    must have cscs card must have asbestos awarene...
 Name: FullDescription, Length: 60000, dtype: object,
 0    we currently have a vacancy for an hr project ...
 1    a web developer opportunity has arisen with an...
 Name: FullDescription, dtype: object)

In [106]:
# ...   exchange every word from data with a number (TF-IDF)   ...
# ...   TF-IDF = (term frequency) * (inverse document frequency) ...
vectorizer = TfidfVectorizer(min_df=5)  # min_df = 5 means "ignore terms that appear in less than 5 documents".

X_train_vec = vectorizer.fit_transform(X_train['FullDescription'])
X_test_vec = vectorizer.transform(X_test['FullDescription'])

X_train_vec, X_test_vec

(<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
 	with 8365759 stored elements in Compressed Sparse Row format>,
 <2x22861 sparse matrix of type '<class 'numpy.float64'>'
 	with 300 stored elements in Compressed Sparse Row format>)

In [107]:
X_train['LocationNormalized'].fillna('nan', inplace=True)
X_train['ContractTime'].fillna('nan', inplace=True)

X_test['LocationNormalized'].fillna('nan', inplace=True)
X_test['ContractTime'].fillna('nan', inplace=True)

X_train.head(), X_test.head()

(                                     FullDescription LocationNormalized  \
 0  international sales manager london     k      ...             London   
 1  an ideal opportunity for an individual that ha...             London   
 2  online content and brand manager   luxury reta...  South East London   
 3  a great local marketleader is seeking a perman...            Dereham   
 4  registered nurse   rgn  nursing home for young...   Sutton Coldfield   
 
   ContractTime  
 0    permanent  
 1    permanent  
 2    permanent  
 3    permanent  
 4          nan  ,
                                      FullDescription LocationNormalized  \
 0  we currently have a vacancy for an hr project ...      Milton Keynes   
 1  a web developer opportunity has arisen with an...         Manchester   
 
   ContractTime  
 0     contract  
 1    permanent  )

In [108]:
# преобразовать номинальные признаки в бинарные
enc = DictVectorizer()

X_train_categ = enc.fit_transform(X_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(X_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_train_categ , X_test_categ

(<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
 	with 120000 stored elements in Compressed Sparse Row format>,
 <2x1766 sparse matrix of type '<class 'numpy.float64'>'
 	with 4 stored elements in Compressed Sparse Row format>)

In [113]:
X_train = hstack([X_train_vec, X_train_categ])
X_test = hstack([X_test_vec, X_test_categ])

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [114]:
# обучение

reg = Ridge(alpha=1, random_state=241)
reg.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

In [117]:
prediction = reg.predict(X_test)
prediction

array([56581.31152328, 37133.54332311])